### Part 1. Create an ETL Pipeline for Pre-Processing files using Cassandra DB

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# CHECK CURRENT WORKING DIRECTORY
print(os.getcwd())

# GET CURRENT FOLDER AND SUBFOLDER EVENT DATA
filepath = os.getcwd() + '/event_data'

# CREATE A FOR LOOP THAT CREATES A LIST OF FILES AND COLLECT EACH FILEPATH
for (root, dirs, files) in os.walk(filepath, topdown=False):
    file_path_list = glob.glob(os.path.join(root,'*'))

/Users/brad/Programming/Learning/DataEngineering/01-DataModeling/CassandraDataModeling/ProjectWorkspace


In [3]:
# INITIATING AN EMPTY LIST OF ROWS THAT WILL BE GENERATED FROM EACH FILE
full_data_rows_list = [] 
    
for f in file_path_list:

# READ THE CSV FILE
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:  
        csvreader = csv.reader(csvfile) 
        next(csvreader)
       
        for line in csvreader:
            full_data_rows_list.append(line) 

# CREATE CSV FILE TO INSERT THE DATA INTO
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### NOW WE CAN WORK WITH THE NEWLY CREATED CSV FILE IN OUR DIRECTORY. 
IT CONTAINS THE FOLLOWING COLUMNS:
   - artist
   - firstName
   - gender
   - itemInSession
   - lastName
   - length
   - level
   - location
   - sessionId
   - song
   - userId
   
   BELOW IS A SCREENSHOT OF ITS APPEARANCE
    ![caption](DataPreview.jpg)

In [6]:
# CREATE THE CLUSTER, SESSION AND INSTANTIATE PRETTYTABLE
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()
x = PrettyTable()
y = PrettyTable()
z = PrettyTable()

In [7]:
# CREATE THE KEYSPACE
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkify 
        WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

In [8]:
# SET THE KEYSPACE
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

---
#### Query 1:
- **This table and query will return the following:** artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

    - **Table Name:** session_item
    - **Primary Key Columns:** sessionId, itemInSession
    - **Cluster Column:** 
    - **Columns in the table will be:** aritist, song, length, sessionId and itemInSession

In [10]:
# CREATE TABLE SESSION_ITEM
query1 = """CREATE TABLE IF NOT EXISTS session_item (sessionId int, itemInSession int, 
artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"""

try:
    session.execute(query1)
except Exception as e:
    print(e)
                

In [11]:
# CREATE THE TABLE INSERT STATEMENTS 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query1 = "INSERT INTO session_item (sessionId, itemInSession, artist, song, length) "
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s) "
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])) )

In [12]:
# SELECT THE DATA AND VERIFY IS HAS BEEN INSERTED INTO EACH TABLE
query1 = "select artist, song, length from session_item WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    x.field_names = ['Artist', 'Song', 'Length']
    x.add_row([row.artist, row.song, row.length])
    print(x)

+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


---
#### Query 2:
- This table and query will return the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

    - Table Name: user_session
    - Primary Key Columns: userId, sessionId
    - Cluster Column: itemInSession
    - Columns in the table will be: itemInSession, artist, song, firstName and lastName

In [13]:
# CREATE TABLE USER_SESSION
query2 = """CREATE TABLE IF NOT EXISTS user_session
    (userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, 
    PRIMARY KEY((userId, sessionId), itemInSession))"""

try:
    session.execute(query2)
except Exception as e:
    print(e)

In [14]:
# CREATE THE TABLE INSERT STATEMENTS 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query2 = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName  ) "
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s) "
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4] ) )

In [15]:
# SELECT THE DATA AND VERIFY IS HAS BEEN INSERTED INTO EACH TABLE
query = "SELECT itemInSession, artist, song, firstName, lastName \
        FROM user_session WHERE userId = 10 AND sessionId = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:  
    y.field_names = ['itemInSession', 'Artist', 'Song', 'First Name', 'Last Name']
    y.add_row([row.iteminsession, row.artist, row.song, row.firstname, row.lastname])
print(y)  

+---------------+-------------------+------------------------------------------------------+------------+-----------+
| itemInSession |       Artist      |                         Song                         | First Name | Last Name |
+---------------+-------------------+------------------------------------------------------+------------+-----------+
|       0       |  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|       1       |    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
|       2       | Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|       3       |   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+---------------+-------------------+------------------------------------------------------+------------+-----------+


---
### Query 3:
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

- Table Name: app_history
- Primary Key Columns: song
- Cluster Column: userId
- Columns in the table will be: song, firstName and lastName

In [16]:
# CREATE TABLE APP_HISTORY
query3 = """CREATE TABLE IF NOT EXISTS app_history
    (song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId))"""

try:
    session.execute(query3)
except Exception as e:
    print(e)

In [17]:
# CREATE THE TABLE INSERT STATEMENTS 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query3 = "INSERT INTO app_history (song, userId, firstName, lastName) "
        query3 = query3 + " VALUES (%s, %s, %s, %s) "
        session.execute(query3, (line[9], int(line[10]), line[1], line[4] ) ) 

In [18]:
# SELECT THE DATA AND VERIFY IS HAS BEEN INSERTED INTO EACH TABLE
query3 = "SELECT firstName, lastName \
        FROM app_history WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    z.field_names = ['First Name', 'Last Name']
    z.add_row([row.firstname, row.lastname])
print(z)  

+------------+-----------+
| First Name | Last Name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


---
#### Drop the tables before closing out the sessions

In [19]:
try:
    session.execute('Drop table session_item')
except Exception as e:
    print(e)

errors={'127.0.0.1:9042': 'Client request timeout. See Session.execute[_async](timeout)'}, last_host=127.0.0.1:9042


In [20]:
try:
    session.execute('Drop table user_session')
except Exception as e:
    print(e)

In [21]:
try:
    session.execute('Drop table app_history')
except Exception as e:
    print(e)

errors={'127.0.0.1:9042': 'Client request timeout. See Session.execute[_async](timeout)'}, last_host=127.0.0.1:9042


#### Close the session and cluster connection

In [22]:
session.shutdown()
cluster.shutdown()